# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [ ]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [ ]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [ ]:
citations.show(5)

In [ ]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [ ]:
patents.show(5)

In [ ]:
patent_state_map = patents.select(
    col("PATENT").alias("patent_id"),
    col("POSTATE").alias("state")
).filter(col("state").isNotNull())

citing_with_state = citations.join(
    patent_state_map, 
    citations.CITING == patent_state_map.patent_id
).select(
    col("CITING"), 
    col("CITED"), 
    col("state").alias("citing_state")
)

full_state_match = citing_with_state.join(
    patent_state_map, 
    citing_with_state.CITED == patent_state_map.patent_id
).select(
    col("CITING"),
    col("CITED"),
    col("citing_state"),
    col("state").alias("cited_state")
)

self_state_only = full_state_match.filter(
    col("citing_state") == col("cited_state")
)

citation_counts = self_state_only.groupBy("CITING").count()

final_output = patents.join(
    citation_counts, 
    patents.PATENT == citation_counts.CITING, 
    "inner"
).orderBy(col("count").desc())

final_output.show(10)